# Secure Multi-Party Computation (SMPC)

[SMPC](https://en.wikipedia.org/wiki/Secure_multi-party_computation) is the backbone of encrypted, federated deep-learning. SMPC is exactly the way it sounds; it makes use of multiple workers - remote parties in a network - as well as a trusted third party worker that provides the encryption.

SMPC protects the privacy of data held by each partyin the network from all the other workers, while maintaing the ability to securely evaluate a global function F across the network.

SMPC works without keys like the ones used for SSH that you may be familiar with. Encrytion keys are not necessay since values are split into *shares* which are distributed amongst the workers. Therefore, in order to decrypt a value all of the workers must agree to put it back together.

Here I will dive into the basics of SMPC, but for a more thorough tutorial, please visit [Morten Dahl's fantastic tutorial on SMPC](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/)

## Goals of SMPC:
1. **Correctness**:
2. **Input Privacy**:

**Please Note**, this is a crash course in SMPC, and for more in depth explanations please see the resources I have referenced while developing this notebook:

- **Morten Dahl's SMPC **
- [Wikipedia: Secure multi-party computation](https://en.wikipedia.org/wiki/Secure_multi-party_computation)
- [PySyft SMPC Tutorial](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2009%20-%20Intro%20to%20Encrypted%20Programs.ipynb)
- [Boston University Article](https://www.bu.edu/articles/2019/secure-multiparty-computation/)
- [Ran Cohen's Lecture](https://www.cs.tau.ac.il/~iftachh/Courses/Seminars/MPC/Intro.pdf)

## Encryption

Encryption is done on the basis of a large number, *Q*. Generally speaking we want *Q* to be a really large prime number. Why? We will get to that in a second. For now, we will pretend that Q is a small numer `Q = 10` for the purpose of this example. We will also be splitting each value into three shares, as if there were three workers/members in our network.

### Additive Secret Sharing
Additive secret sharing is a subfield of cryptography that allows us to not have to rely on a trusted third party aggregator.
We want to aggregate numbers while numbers are encrypted so that no individual entity will see what the encrypted number actually is.
Additive secret sharing allows us to sum encrypted numbers together without revealing what the inputs are.
This technique is actually computationally efficient, which is nice.

One thing to note is that without an extra step, we actually leak some information. For example, if `share_a=2` and we know we only have positive integers, then `decrypted_x > 2`. To fix this potential information leakage, we use very large number Q. This now gives us an infinite number of possibilites for `share_a` and `share_b`, that could map to the correct output value.

### Basics of Encrypted, SMPC, Arithmetic
Example using two workers communicating together, so that we can see how encryption works under the hood.

We want to encode the number `5`, and the number `1` and carry out arithmetic using the encrypted numbers. To do this we use a `field` variable (herein represented by `Q`).

In [56]:
Q=10
x=5

share_a_x = 8
share_b_x = Q - share_a_x + x

In [57]:
(share_a_x + share_b_x) % Q #the modulus is dividing and giving remainder.

5

Note, there is an infinite number of ways to encode the number 5, since there are infinite numbes `%10` that returns 5!

This is also the basis of encrypted arithmetic. Let's say we have two numbers 5,1 that encrypt into 15 and 11 using the example above.

If we add them together and take the modulus, the decryption logic still holds.

In [52]:
print(15 + 11, (15+11)%Q)

26 6


When we add two numbers that are within a field Q, then the remainder also adds up in the process, but the field (Q) piece is equivalent to 0, because it modulus = 0. Any number >=Q, moduluses to 0, which is again why we want a large Q! *We cannot represent numbers larger than Q in SMPC.*

**Subtraction**
The same logic works for subtraction as well. 

In [53]:
y=1
share_a_y = 9
share_b_y = Q - share_a_y + y

In [54]:
(share_a_y + share_b_y) % Q #the modulus is dividing and giving remainder.

1

In [58]:
(share_a_x + share_b_x) % Q - (share_a_y + share_b_y) % Q  #subtract decrypted numbers

4

In [59]:
((share_a_x + share_b_x) - (share_a_y + share_b_y)) % Q   #subtract encrypted numbers and decrypt

4

In [60]:
((share_a_x - share_a_y) + (share_b_x-share_b_y)) % Q 
#subtract encrypted numbers on each remote worker first before decrypting
#we also dont have to decrypt it, each remote worker can keep the arithmetic output in their own share.

4

All of these operations are equivalent! That means we can actually perform encrypted, federated arithmetic using secure multi-party computation!
These same properties - the fact that all operations are done within the field Q and Q allows us to encrypt values - apply for addition and subtraction.

Handle **negative number reperesentation** in `encode()` or `decode()`


##### Multiplication
Multiplication is quite straightforward if you understand addition, since multiplication simply builds on addition.
`2+2=4`, `2*2=4` etc. Note that this technique allows for multiplying encrypted numbers with non-encrypted numbers! This would be computationally beneficial in scenarios where not all numbers need to be encrypted.

### Additive Secret Sharing

Here I build out more intricate encryption, and decryption functionalities.

In [80]:
import random
def encrypt(x, n_shares=3):
    '''
    Use  SMPC to encrypt a number
    
    Inputs: x
    Outputs: 
    '''
    
    #generate first two shares randomly
    shares=list()
    for i in range(n_shares-1):
        shares.append(random.randint(0,Q)) #each workers share is entirely random
        #this is also why we want a large Q value because that allows better encryption 
        #and allows encryption of larger values

    final_share = Q - (sum(shares)%Q) + x #sum of shares modulus Q shows us how far the sum is away from Q

    shares.append(final_share)
    return tuple(shares)

I have found the calculation of the final share to be tricky to comprehend. Think about it this way though:

If `Q = 10`, and all the randomly assigned shares till now add up to `6`, then `sum(shares)%Q = 6`. Then `Q - (sum(shares)%Q) = 4` which means that after considering all but the last share we are still 4 away from the value Q. Therefore we want to add `4` to be part of the final share since all the other numbers will be cancelled out in the decryption step where we modulus the sum of shares by `10` and all that is left is the value we wanted to encrypt originally, `x`. If this is still tricky, work through some of the examples of encrypted arithmetic above. 

In [77]:
def decrypt(shares):
    '''Additive Secret Sharing Decryption'''
    return sum(shares) % Q

In [82]:
Q = 10
x=5
_x = encrypt(x)
_x

(1, 3, 11)

In [84]:
Q=782372130292931827320302390213721837
_x = encrypt(x)
_x
#nice!

(414188986592025917459316057488415215,
 215916764030367714056052269452366058,
 152266379670538195804934063272940569)

Note that we can only represent numbers as large as Q... so if we try to encrypt and decrypt 11, while `Q=10` we get the equivalent of an overflow error.

In [87]:
Q=10
print(encrypt(11), decrypt(encrypt(11)))#why we want really large number for Q

(7, 2, 12) 1


In [35]:
def add(a, b):
    c = list()
    
    for i in range(len(a)):
        c.append((a[i]+b[i])%Q)
        
    return tuple(c)

In [ ]:
def sub(a, b):
    c = list()
    
    for i in range(len(a)):
        c.append((a[i]-b[i])%Q)
        
    return tuple(c)

In [90]:
def mul(a, b):
    c = list()
    
    for i in range(len(a)):
        c.append((a[i]*b)%Q)
        
    return tuple(c)

In [91]:
decrypt(mul(encrypt(2),4))

8

In [36]:
decrypt(add(encrypt(3), encrypt(4)))

7

### Add Support for Decimals - Fixed Precision Encoding

PySyft actually has a FixedPrecisionTensor implementation that handles this for us when we call `tensor.fix_precision()`

In [100]:
BASE = 10 #numbers of base 10
PRECISION = 4 #store 4 decimals

Q = 231323129859892723107

def encode(x_dec):
    '''Fixed Precision Encoding. Input: float'''
    
    return int(x_dec * (BASE ** PRECISION))%Q #multiple decimal by 10000 and convert to integer

In [23]:
encode(0.99)

9900

In [25]:
def decode(x_fp):
    '''Divide By Fixed Precision threshold, this first part of if, else helps us account for negative numbers'''
    return (x_fp if x_fp <= Q/2 else x_fp - Q)/ BASE**PRECISION

**Note** that `decode()` shows we are splitting our field (Q) into two sections. [0,Q/2] represents positive numbers, (Q/2,inf] represents negative numbers. This is due to our fixed precision encoding rather than SMPC

In [26]:
decode(9900)

0.99

In [27]:
encode(-0.99) #wraps around to other side of int.

231323129859892713207

In [28]:
decode(231323129859892713207)

-0.99

### Processing Flow:

How to process numbers to ensure correct logic in SMPC and encrypted computation

1. Encode (fixed precision)
2. Encrypt
3. Arithmetic
4. Decrypt
5. Decode

In [102]:
Q=193387283123
decrypt(add(encrypt(encode(0.99)),encrypt(encode(1))))

19900

In [103]:
decode(19900)

1.99